![image](google-header.png)

## **Google Search Analysis**

So in the previous workbook we explored getting the data from Google. Awesome! Lets take a step further and see if we can sift through those results and find the outliers.

## **Overview**
The following cells will do some data exploration, basic cleanup, some feature engineering, and then clustering and text analysis.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import seaborn as sns
import spacy
from spacy import displacy



RAW_DATA = 'data'

/Users/matteads/anaconda3/envs/dev/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/matteads/anaconda3/envs/dev/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/matteads/anaconda3/envs/dev/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/matteads/anaconda3/envs/dev/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/matteads/anaconda3/envs/dev/lib/python3.7/importlib/_bootstra

In [75]:
nlp = spacy.load('en_core_web_md')

doc = nlp('Hi my name is Matt I like to use Python tool and data science. I am currently an employee at Vanguard Corporation where I joined back in November 2020. My previous emplyer KPMG I was a data scientist. Prior to KPMG I attended University of Saint Joseph\'s. This is a $100 Million.')

In [78]:
displacy.render(doc, style='ent', jupyter=True)